# Intro
Here I will show a way to build a unified mask with borders between touching nuclei for the traning data.
I am new to ML and from my understanding, making a one unified image of masks should make it easier to train the U-Net model. The problem with one unified image of masks is to distinguish between nuclei that are touching each other.
One way to get around that is by building a border between those nucleis and training the network.

Insperation for this code I took from the Kjetil Åmdal-SævikKeras notebook "U-Net starter - LB 0.277": [https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277](https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277)

In [7]:
import os
from tqdm import tqdm
import numpy as np
import random
import matplotlib as plt

from skimage import feature
from skimage.io import imread, imshow
from skimage.transform import SimilarityTransform, warp

%matplotlib inline

In [4]:
TRAIN_PATH = '../input/stage1_train/'

train_ids = next(os.walk(TRAIN_PATH))[1]

Lets take a random image and see whats the problem:

In [11]:
def create_unified_mask(image_path, mask_shape):
    
    final_mask = np.zeros(mask_shape)
    
    masks = next(os.walk(TRAIN_PATH + image_path + '/masks/'))[2]
        
    for mask in masks:
        m = imread(TRAIN_PATH + image_path + '/masks/' + mask)        
        final_mask = np.maximum(final_mask, m)
    
    return final_mask

In [13]:
image_path = train_ids[random.randint(0, len(train_ids))]
img = imread(TRAIN_PATH + image_path + '/images/' + image_path + '.png')
unified_mask = create_unified_mask(image_path, img.shape[0:2])

imshow(unified_mask)

We can clearly see the that there are nuclei that are touching each other with no easy way to separate them.

A simple way to overcome this is by shifting each mask and comparing it to other masks and zero the pixels that overlap.

In [18]:
def create_unified_mask_with_borders(image_path, mask_shape):
    
    final_mask = np.zeros(mask_shape)
    
    masks = next(os.walk(TRAIN_PATH + image_path + '/masks/'))[2]
        
    for mask in masks:
        m = imread(TRAIN_PATH + image_path + '/masks/' + mask) / 255
        
        image_left = warp(m, SimilarityTransform(translation=(2, 0)))
        image_right = warp(m, SimilarityTransform(translation=(-2, 0)))
        image_up = warp(m, SimilarityTransform(translation=(0, 2)))
        image_down = warp(m, SimilarityTransform(translation=(0, -2)))
        
        border = final_mask + image_left == 2
        border = np.logical_or(border, final_mask + image_right == 2)
        border = np.logical_or(border, final_mask + image_up == 2)
        border = np.logical_or(border, final_mask + image_down == 2)
        
        final_mask = np.maximum(final_mask, m)
        final_mask[border] = 0
    
    return final_mask

In [19]:
unified_mask_with_border = create_unified_mask_with_borders(image_path, img.shape[0:2])

imshow(unified_mask_with_border)

The problem in this solution is that we introduce an error to the training data which is generaly bad but on the other hand it is a fast and easy way to start working with U-Net.